In [1]:
!pip install kaggle==1.5.12

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json  # Set permissions

In [3]:
!mkdir datasets

mkdir: cannot create directory ‘datasets’: File exists


In [4]:
!kaggle datasets download -d mozillaorg/common-voice -p /content/datasets --force

  3% 345M/12.0G [00:17<10:06, 20.7MB/s]
User cancelled operation


In [5]:
%cd datasets

/content/datasets


In [6]:
!unzip common-voice.zip

Archive:  common-voice.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of common-voice.zip or
        common-voice.zip.zip, and cannot find common-voice.zip.ZIP, period.


In [7]:
import pandas as pd

In [8]:
dev_df = pd.read_csv('cv-valid-dev.csv')
dev_df.head()

,filename,text,up_votes,down_votes,age,gender,accent,duration
0,cv-valid-dev/sample-000000.mp3,be careful with your prognostications said the...,1,0,NaN,NaN,NaN,NaN
1,cv-valid-dev/sample-000001.mp3,then why should they be surprised when they se...,2,0,NaN,NaN,NaN,NaN
2,cv-valid-dev/sample-000002.mp3,a young arab also loaded down with baggage ent...,2,0,NaN,NaN,NaN,NaN
3,cv-valid-dev/sample-000003.mp3,i thought that everything i owned would be des...,3,0,NaN,NaN,NaN,NaN
4,cv-valid-dev/sample-000004.mp3,he moved about invisible but everyone could he...,1,0,fourties,female,england,NaN


In [9]:
dev_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4076 entries, 0 to 4075
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   filename    4076 non-null   object 
 1   text        4076 non-null   object 
 2   up_votes    4076 non-null   int64  
 3   down_votes  4076 non-null   int64  
 4   age         1528 non-null   object 
 5   gender      1540 non-null   object 
 6   accent      1350 non-null   object 
 7   duration    0 non-null      float64
dtypes: float64(1), int64(2), object(5)
memory usage: 254.9+ KB


In [ ]:
!pip install pydub

In [ ]:
from IPython.display import Audio, display
import os

def display_audio(audio_path):
  display(Audio(audio_path))

audio_directory = 'cv-valid-dev/cv-valid-dev'
audio_files = [audio for audio in os.listdir(audio_directory)]

for audio_file in audio_files[:5]:
  audio_path = os.path.join(audio_directory, audio_file)
  display_audio(audio_path)


In [ ]:
dev_df['down_votes'].value_counts()

In [ ]:
dev_df[dev_df['down_votes'] == 0][:10]

In [ ]:
audio_path = os.path.join(audio_directory, "sample-000000.mp3")
display_audio(audio_path)

 => It seems that the difference between upvotes and downvotes doesn't relate to the quality of audios

**Preprocessing steps**



In [ ]:
dev_df = dev_df.drop(columns=dev_df.columns[dev_df.columns.get_loc('up_votes') : dev_df.columns.get_loc('duration') + 1])

In [ ]:
dev_df.head()

In [ ]:
import torch
import torchaudio
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
train_df, test_df = torch.utils.data.random_split(dev_df, [0.8, 0.2])

In [ ]:
def preprocess_for_rnnt_torchaudio(file_path, target_sr=16000, max_duration=5):
    """GPU-accelerated preprocessing pipeline for RNN-T using TorchAudio"""

    # 1. Load audio directly to GPU (PyTorch 2.0+)
    waveform, sr = torchaudio.load(file_path)
    waveform = waveform.to(device)

    # 2. Resample if needed
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)

    # 3. Trim silence (VAD)
    waveform = torchaudio.functional.vad(waveform, sample_rate=target_sr, trigger_level=20)

    # 4. Peak normalization (GPU)
    waveform = torch.nn.functional.normalize(waveform, dim=-1)

    # 5. Fixed-length padding/cropping
    max_samples = target_sr * max_duration
    if waveform.size(-1) > max_samples:
        waveform = waveform[..., :max_samples]
    else:
        pad_amount = max_samples - waveform.size(-1)
        waveform = torch.nn.functional.pad(waveform, (0, pad_amount))

    # 6. Extract Log-Mel features (GPU)
    mel_transform = torchaudio.transforms.MelSpectrogram(
        sample_rate=target_sr,
        n_fft=400,
        hop_length=160,
        n_mels=80,
    ).to(device)

    mel_spec = mel_transform(waveform)
    log_mel = torch.log(mel_spec + 1e-6)  # (1, 80, T)

    return log_mel.squeeze(0).T  # (T, 80)

In [ ]:
audio_directory = 'cv-valid-dev'
audio_files = [train_df.dataset.loc[train_df.indices[i], 'filename'] for i in range(len(train_df))]

In [ ]:
features = []
for audio in audio_files:
  audio_feature = preprocess_for_rnnt_torchaudio(os.path.join(audio_directory, audio))
  features.append(audio_feature)

In [ ]:
texts = train_df["text"].tolist()

In [ ]:
texts[:5]

In [ ]:
import re

In [ ]:
def clean_text(text):
    # Keep original case - REMOVED .lower()
    text = text.strip()

    # Handle apostrophes/contractions carefully
    text = re.sub(r"([!\"'#$%&()*\+,-./:;<=>?@\\\[\]^_`{|}~])", r" \1 ", text)
    text = re.sub("[^A-Za-z0-9]+", " ", text)
    text = re.sub(" +", " ", text)

    return text

In [ ]:
# Apply cleaning
cleaned_texts = [clean_text(t) for t in texts if isinstance(t, str)]

In [ ]:
cleaned_texts[:5]

In [ ]:
from tokenizers import Tokenizer, models, trainers
from tokenizers import pre_tokenizers

In [ ]:
# Initialize with byte-level BPE (better for names/contractions)
tokenizer = Tokenizer(models.BPE(unk_token="<unk>"))

# Simpler pre-tokenizer (preserve apostrophes)
tokenizer.pre_tokenizer = pre_tokenizers.Sequence([
    pre_tokenizers.Whitespace(),
    pre_tokenizers.Split("'", behavior="isolated")  # Special handling for apostrophes
])

# Trainer with larger vocab
trainer = trainers.BpeTrainer(
    special_tokens=["<pad>", "<unk>", "<bos>", "<eos>"],
    vocab_size=3800,  # Increased for better word coverage
    min_frequency=3,
    initial_alphabet=pre_tokenizers.ByteLevel.alphabet()  # Better for special characters
)

# Train on properly cleaned text
tokenizer.train_from_iterator([clean_text(t) for t in texts], trainer)
tokenizer.save("cv_tokenizer.json")

In [ ]:
# Load tokenizer
tokenizer = Tokenizer.from_file("cv_tokenizer.json")

# Test encoding
sample_text = "Dr O'Neill 's patient said We 'll check again tomorrow"
encoding = tokenizer.encode(sample_text.lower())
print("Tokens:", encoding.tokens)
print("IDs:", encoding.ids)

In [ ]:
class Encoder(torch.nn.Module):
    def __init__(self, input_dim=80, hidden_dim=256, num_layers=3):
        super().__init__()
        self.lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            bidirectional=True,
            batch_first=True,
        )
        self.linear = torch.nn.Linear(hidden_dim * 2, hidden_dim)

    def forward(self, x):
        x, _ = self.lstm(x)  # (B, T, 2*H)
        x = self.linear(x)    # (B, T, H)
        return x

In [ ]:
class Decoder(torch.nn.Module):
    def __init__(self, vocab_size, embed_dim=128, hidden_dim=256):
        super().__init__()
        self.embed = torch.nn.Embedding(vocab_size, embed_dim)
        self.lstm = torch.nn.LSTM(embed_dim, hidden_dim, batch_first=True)

    def forward(self, y):
        y = self.embed(y)     # (B, U, E)
        y, _ = self.lstm(y)   # (B, U, H)
        return y

In [ ]:
class JointNetwork(torch.nn.Module):
    def __init__(self, hidden_dim, vocab_size):
        super().__init__()
        self.linear = torch.nn.Linear(hidden_dim, vocab_size)

    def forward(self, h_enc, h_dec):
        # h_enc: (B, T, H), h_dec: (B, U, H)
        h_enc = h_enc.unsqueeze(2)  # (B, T, 1, H)
        h_dec = h_dec.unsqueeze(1)  # (B, 1, U, H)
        out = torch.tanh(h_enc + h_dec)  # (B, T, U, H)
        return self.linear(out)  # (B, T, U, V)

In [ ]:
class RNNTransducer(torch.nn.Module):
    def __init__(self, vocab_size, encoder_dim=256, decoder_dim=256):
        super().__init__()
        self.encoder = Encoder(hidden_dim=encoder_dim)
        self.decoder = Decoder(vocab_size, hidden_dim=decoder_dim)
        self.joint = JointNetwork(encoder_dim, vocab_size)

    def forward(self, x, y):
        h_enc = self.encoder(x)  # (B, T, H)
        h_dec = self.decoder(y)  # (B, U, H)
        logits = self.joint(h_enc, h_dec)  # (B, T, U, V)
        return logits

In [ ]:
# Continue from where the notebook left off

# First, let's prepare the dataset class
class AudioTextDataset(torch.utils.data.Dataset):
    def __init__(self, features, texts, tokenizer, max_text_length=100):
        self.features = features
        self.texts = texts
        self.tokenizer = tokenizer
        self.max_text_length = max_text_length

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        # Get audio features
        audio_feature = self.features[idx]

        # Get and tokenize text
        text = self.texts[idx]
        encoding = self.tokenizer.encode(text)
        text_ids = encoding.ids

        # Add BOS and EOS tokens
        text_ids = [tokenizer.token_to_id("<bos>")] + text_ids + [tokenizer.token_to_id("<eos>")]

        # Pad text sequence
        if len(text_ids) < self.max_text_length:
            pad_amount = self.max_text_length - len(text_ids)
            text_ids = text_ids + [tokenizer.token_to_id("<pad>")] * pad_amount
        else:
            text_ids = text_ids[:self.max_text_length]

        return {
            'audio': audio_feature,
            'text': torch.tensor(text_ids, dtype=torch.long)
        }

In [ ]:
# Create datasets
train_dataset = AudioTextDataset(
    features=[features[i] for i in train_df.indices],
    texts=[cleaned_texts[i] for i in train_df.indices],
    tokenizer=tokenizer
)

test_dataset = AudioTextDataset(
    features=[features[i] for i in test_df.indices],
    texts=[cleaned_texts[i] for i in test_df.indices],
    tokenizer=tokenizer
)

In [ ]:
# Create data loaders
def collate_fn(batch):
    # Pad audio features to same length
    audio_features = [item['audio'] for item in batch]
    max_audio_len = max(f.size(0) for f in audio_features)

    padded_audio = []
    for f in audio_features:
        pad_amount = max_audio_len - f.size(0)
        if pad_amount > 0:
            f = torch.nn.functional.pad(f, (0, 0, 0, pad_amount), value=0)
        padded_audio.append(f)

    audio_tensor = torch.stack(padded_audio)
    text_tensor = torch.stack([item['text'] for item in batch])

    return {
        'audio': audio_tensor.to(device),
        'text': text_tensor.to(device)
    }

batch_size = 16
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn
)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn
)

In [ ]:
# Initialize model
vocab_size = tokenizer.get_vocab_size()
model = RNNTransducer(vocab_size).to(device)

# Loss function (RNN-T loss)
def rnnt_loss(logits, targets, input_lengths, target_lengths, blank=0):
    # logits: (B, T, U, V)
    # targets: (B, U)
    # input_lengths: (B,) - length of each audio sequence
    # target_lengths: (B,) - length of each text sequence

    # Convert to log-probs
    log_probs = torch.nn.functional.log_softmax(logits, dim=-1)

    # Transpose for torchaudio's expectation (T, B, U, V)
    log_probs = log_probs.permute(1, 0, 2, 3)

    # Compute RNN-T loss
    loss = torchaudio.functional.rnnt_loss(
        log_probs,
        targets,
        input_lengths,
        target_lengths,
        blank=blank,
        reduction='mean'
    )

    return loss


In [ ]:
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

# Training loop
def train_epoch(model, dataloader, optimizer):
    model.train()
    total_loss = 0

    for batch in dataloader:
        audio = batch['audio']
        text = batch['text']

        # Get input/target lengths
        input_lengths = torch.tensor([a.size(0) for a in audio], device=device)
        target_lengths = torch.tensor([(t != tokenizer.token_to_id("<pad>")).sum() for t in text], device=device)

        # Forward pass
        optimizer.zero_grad()
        logits = model(audio, text[:, :-1])  # Exclude EOS token for decoder input

        # Compute loss
        loss = rnnt_loss(
            logits,
            text[:, 1:],  # Exclude BOS token for targets
            input_lengths,
            target_lengths - 1  # Subtract 1 because we excluded BOS
        )

        # Backward pass
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
# Evaluation loop
def evaluate(model, dataloader):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for batch in dataloader:
            audio = batch['audio']
            text = batch['text']

            input_lengths = torch.tensor([a.size(0) for a in audio], device=device)
            target_lengths = torch.tensor([(t != tokenizer.token_to_id("<pad>")).sum() for t in text], device=device)

            logits = model(audio, text[:, :-1])

            loss = rnnt_loss(
                logits,
                text[:, 1:],
                input_lengths,
                target_lengths - 1
            )

            total_loss += loss.item()

    return total_loss / len(dataloader)

In [ ]:
# Training
num_epochs = 10
for epoch in range(num_epochs):
    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss = evaluate(model, test_loader)

    print(f"Epoch {epoch+1}/{num_epochs}")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")


In [ ]:
# Inference function
def predict(model, audio_feature, tokenizer, max_decode_len=100):
    model.eval()

    # Add batch dimension
    audio_feature = audio_feature.unsqueeze(0).to(device)
    input_length = torch.tensor([audio_feature.size(1)], device=device)

    # Initialize with BOS token
    decoded = [tokenizer.token_to_id("<bos>")]

    with torch.no_grad():
        h_enc = model.encoder(audio_feature)  # (1, T, H)

        for _ in range(max_decode_len):
            # Get last predicted token
            last_token = torch.tensor([decoded[-1]], device=device).unsqueeze(0)

            # Decoder step
            h_dec = model.decoder(last_token)  # (1, 1, H)

            # Joint network
            logits = model.joint(h_enc, h_dec)  # (1, T, 1, V)
            log_probs = torch.nn.functional.log_softmax(logits.squeeze(2), dim=-1)  # (1, T, V)

            # Sum over time (approximate)
            scores = log_probs.sum(1)  # (1, V)
            next_token = scores.argmax(-1).item()

            # Stop if EOS is predicted
            if next_token == tokenizer.token_to_id("<eos>"):
                break

            decoded.append(next_token)

    # Convert to text
    tokens = tokenizer.decode(decoded[1:])  # Skip BOS
    return tokens


In [ ]:
# Test inference
test_idx = 0
test_audio = features[test_idx]
test_text = cleaned_texts[test_idx]

predicted_text = predict(model, test_audio, tokenizer)
print(f"Original: {test_text}")
print(f"Predicted: {predicted_text}")